In [2]:
import os 
import string 
import json 
import pandas as pd
import numpy as np

In [11]:
data_hosting = list()
data = list()
index_id = 0
with open('C:\logs\logs\log2020042214.log', 'r') as f:
    for cnt, line in enumerate(f):
        obj = json.loads(line)        
        properties = obj['Properties']
        if obj['Properties']['SourceContext'] == "Microsoft.AspNetCore.Hosting.Diagnostics" and "HostingRequestFinishedLog" in properties:
            data_hosting.append( [obj['Timestamp'],                           
                          properties['ElapsedMilliseconds'],
                          properties['RequestPath'], 
                          properties['RequestId'], 
                          properties['StatusCode'], 
                          properties['ThreadId']   ])

            
        if obj['Properties']['SourceContext'] == "Microsoft.EntityFrameworkCore.Database.Command":                        
            data.append( [index_id, obj['Timestamp'], 
                          properties['commandText'].replace('\n', '' ), 
                          properties['elapsed'].replace(',',''), 
                          properties['RequestPath'], 
                          properties['RequestId'], 
                          properties['ConnectionId'], 
                          properties['ThreadId']   ])

            index_id += 1

df_hosting = pd.DataFrame(data_hosting, columns=['timespan', 'elapsed',  'action', 'request', 'status', 'thread'])
df_hosting[['elapsed']] = df_hosting[['elapsed']].apply(pd.to_numeric)             
df  = pd.DataFrame(data, columns=['index', 'timespan', 'text', 'elapsed',  'action', 'request', 'connection', 'thread'])
df[['elapsed']] = df[['elapsed']].apply(pd.to_numeric) 

display(df_hosting.info())

display(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2447 entries, 0 to 2446
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   timespan  2447 non-null   object 
 1   elapsed   2447 non-null   float64
 2   action    2447 non-null   object 
 3   request   2447 non-null   object 
 4   status    2447 non-null   int64  
 5   thread    2447 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 114.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106064 entries, 0 to 106063
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   index       106064 non-null  int64 
 1   timespan    106064 non-null  object
 2   text        106064 non-null  object
 3   elapsed     106064 non-null  int64 
 4   action      106064 non-null  object
 5   request     106064 non-null  object
 6   connection  106064 non-null  object
 7   thread      106064 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 6.5+ MB


None

In [12]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [23]:
actions = df_hosting['action'].unique().tolist()
data_actions = list()
for action in actions:
    temp = df_hosting[df_hosting['action']==action][['action', 'elapsed']].groupby(['action']).agg(['count',np.sum, np.mean, np.std, np.median,
                     np.var, np.min, np.max, percentile(95), percentile(99)])
    per99  = list(temp['elapsed']['percentile_99'])    
    
    count_items  = sum(list(temp['elapsed']['count']))    
    data_actions.append( [ action, sum(per99), count_items] )
    display('{} , {} has cost in api {}'.format(action, count_items, sum(per99)))


'/migrations/restore , 1 has cost in api 57694.081'

'/customers , 198 has cost in api 168.6297030000001'

'/products , 198 has cost in api 74.3606880000001'

'/services , 196 has cost in api 1311.5652400000001'

'/features , 193 has cost in api 5164.570747999998'

...

'/services/3 , 183 has cost in api 7075.149324000004'

'/services/3/reports/daily/series , 183 has cost in api 717.088826000001'

'/services/4 , 183 has cost in api 7191.047600000001'

'/services/4/reports/daily/series , 183 has cost in api 782.0964520000018'

'/sources , 183 has cost in api 364.8607940000007'

In [24]:
for action in sorted(data_actions, key=lambda x: x[1], reverse=True):
    if action[1] >= 1000 and '/migrations/restore' not in action:
        print('reqeuest {} with {} in {}'.format(action[0], action[2], action[1]))
        temp = df[df['action']==action[0]][['text', 'elapsed']].groupby(['text']).agg(['count',
                                                                                       percentile(95), 
                                                                                       percentile(99)])
        
        display(temp)
        #per99  = list(temp['elapsed']['percentile_99'])            
        
        
        
    
    
    
    

reqeuest /services/4 with 183 in 7191.047600000001


elapsed                \
                                                     count percentile_95   
text                                                                       
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...     366         24.00   
SELECT [i0].[Id], [i0].[Affected], [i0].[Create...    1830         25.00   
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...     366         18.00   
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...   36600         26.00   
SELECT [t].[Id], [t].[Aggregation], [t].[Avatar...     366         27.75   

                                                                  
                                                   percentile_99  
text                                                              
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...         57.10  
SELECT [i0].[Id], [i0].[Affected], [i0].[Create...         89.55  
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...         32.70  
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...         72.00  
SELECT [t].[Id], [t].[Aggregation], [t].[Avatar...         60.95

reqeuest /services/3 with 183 in 7075.149324000004


elapsed                \
                                                     count percentile_95   
text                                                                       
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...     372         24.45   
SELECT [i0].[Id], [i0].[Affected], [i0].[Create...    1842         20.00   
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...     372         24.45   
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...   37035         27.00   
SELECT [t].[Id], [t].[Aggregation], [t].[Avatar...     372         28.45   

                                                                  
                                                   percentile_99  
text                                                              
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...         59.48  
SELECT [i0].[Id], [i0].[Affected], [i0].[Create...         64.13  
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...         69.67  
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...         70.00  
SELECT [t].[Id], [t].[Aggregation], [t].[Avatar...         61.61

reqeuest /features with 193 in 5164.570747999998


elapsed                \
                                                     count percentile_95   
text                                                                       
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...      10        420.85   
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...     183        112.40   
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...   18300         28.00   

                                                                  
                                                   percentile_99  
text                                                              
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...        480.97  
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...        311.36  
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...         80.00

reqeuest /products/2/reports/daily/services/series with 370 in 3942.093019


elapsed                \
                                                     count percentile_95   
text                                                                       
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...     371          34.0   
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...     371          27.0   
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...     371          25.5   
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...     371          29.0   
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...     371          33.0   

                                                                  
                                                   percentile_99  
text                                                              
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...         128.3  
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...          57.1  
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...         101.4  
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...          69.2  
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...         121.2

reqeuest /services with 196 in 1311.5652400000001


elapsed                \
                                                     count percentile_95   
text                                                                       
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...     187          28.0   
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...     187          27.1   
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...      10         142.0   
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...     187          25.7   
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...     187          23.0   
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...     373          42.4   

                                                                  
                                                   percentile_99  
text                                                              
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...         62.68  
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...        164.28  
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...        178.00  
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...         60.78  
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...         41.32  
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...        306.52

In [64]:
data_actions = sorted(data_actions, key=lambda x: x[1], reverse=True) 
for action in data_actions[:3]:
    display('{} has cost in DB {}'.format(action[0], action[1]))
    worst = df[df['action']==action[0]]
    display(worst[['text', 'elapsed']].groupby(['text']).agg(['count', percentile(99)]))    
    
    


'Owlvey.Falcon.API.Controllers.MigrationController.PostBackupAsync (Owlvey.Falcon.API) has cost in DB 2092.3999999999996'

elapsed              
                                                     count percentile_99
text                                                                    
SELECT TOP(2) [a].[Id], [a].[CreatedBy], [a].[C...       2          1.00
SELECT TOP(2) [c].[Id], [c].[Avatar], [c].[Crea...       1         26.00
SELECT TOP(2) [c].[Id], [c].[Avatar], [c].[Crea...       1          1.00
SELECT TOP(2) [f].[Id], [f].[Avatar], [f].[Crea...      13          2.76
SELECT TOP(2) [f].[Id], [f].[Avatar], [f].[Crea...     203          3.00
SELECT TOP(2) [f].[Id], [f].[Avatar], [f].[Crea...     103          3.00
SELECT TOP(2) [f].[Id], [f].[Avatar], [f].[Crea...     206          2.00
SELECT TOP(2) [i].[Id], [i].[Avatar], [i].[Crea...     103          4.00
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...     203          3.00
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...       2          1.00
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...      48          2.00
SELECT TOP(2) [s].[Id], [s].[Aggregation], [s]....      22          2.79
SELECT TOP(2) [s].[Id], [s].[Aggregation], [s]....     203          3.00
SELECT TOP(2) [s].[Id], [s].[Aggregation], [s]....     203          2.00
SELECT TOP(2) [s].[Id], [s].[Aggregation], [s]....      22          6.37
SELECT TOP(2) [s].[Id], [s].[Avatar], [s].[Crea...       2          1.00
SELECT TOP(2) [s].[Id], [s].[Avatar], [s].[Crea...       3          1.98
SELECT TOP(2) [s].[Id], [s].[Avatar], [s].[Crea...      13          2.88
SELECT TOP(2) [s].[Id], [s].[Avatar], [s].[Crea...     103          3.00
SELECT TOP(2) [s].[Id], [s].[CreatedBy], [s].[C...     203          2.00
SELECT [c].[Id], [c].[Avatar], [c].[CreatedBy],...       2         22.78
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...       1          1.00
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...       1          1.00
SELECT [p].[Id], [p].[Avatar], [p].[CreatedBy],...       1         73.00
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...       1          2.00
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...       1          1.00
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...       1          1.00
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...       1          1.00
SELECT [t].[Id], [t].[Avatar], [t].[CreatedBy],...      13          1.88
SELECT [t].[Id], [t].[Avatar], [t].[CreatedBy],...      13          1.88
SELECT [u].[Id], [u].[Avatar], [u].[CreatedBy],...       1          1.00
SET NOCOUNT ON;DECLARE @inserted0 TABLE ([Id] i...       3          6.98
SET NOCOUNT ON;DECLARE @inserted0 TABLE ([Id] i...      10        200.71
SET NOCOUNT ON;DECLARE @inserted0 TABLE ([Id] i...      63        188.56
SET NOCOUNT ON;DELETE FROM [CustomerEntity]WHER...       1        711.00
SET NOCOUNT ON;DELETE FROM [FeatureEntity]WHERE...      13          2.76
SET NOCOUNT ON;DELETE FROM [ServiceEntity]WHERE...      22        133.56
SET NOCOUNT ON;DELETE FROM [SquadFeatureEntity]...       3          1.00
SET NOCOUNT ON;INSERT INTO [AnchorEntity] ([Cre...       2          2.98
SET NOCOUNT ON;INSERT INTO [CustomerEntity] ([A...       1          1.00
SET NOCOUNT ON;INSERT INTO [FeatureEntity] ([Av...      13          2.88
SET NOCOUNT ON;INSERT INTO [IndicatorEntity] ([...     103          1.00
SET NOCOUNT ON;INSERT INTO [ProductEntity] ([Av...       2         29.71
SET NOCOUNT ON;INSERT INTO [ServiceEntity] ([Ag...      22        623.94
SET NOCOUNT ON;INSERT INTO [ServiceMapEntity] (...     203          1.00
SET NOCOUNT ON;INSERT INTO [SourceEntity] ([Ava...      13          1.00
SET NOCOUNT ON;INSERT INTO [SquadEntity] ([Avat...       2          1.00
SET NOCOUNT ON;INSERT INTO [SquadFeatureEntity]...       3          1.00
SET NOCOUNT ON;UPDATE [AnchorEntity] SET [Modif...       2          1.00

'Owlvey.Falcon.API.Controllers.ProductController.ReportSeries (Owlvey.Falcon.API) has cost in DB 1744.1000000000004'

elapsed              
                                                     count percentile_99
text                                                                    
SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[Crea...     495        459.60
SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy],...     495        333.90
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...     495        455.50
SELECT [s].[Id], [s].[Avatar], [s].[CreatedBy],...     495         20.12
SELECT [s].[Id], [s].[CreatedBy], [s].[CreatedO...     495        474.98

'Owlvey.Falcon.API.Controllers.ServiceController.Get (Owlvey.Falcon.API) has cost in DB 1550.9499999999975'

elapsed              
                                                     count percentile_99
text                                                                    
SELECT [s].[Id], [s].[Aggregation], [s].[Avatar...      20       1550.95

In [59]:
worst = df[df['action']=='Owlvey.Falcon.API.Controllers.ProductController.ReportSeries (Owlvey.Falcon.API)']
worst[['text', 'elapsed']].groupby(['text']).agg('max')

pd.set_option('display.max_row', 1000)
''.join(worst['text'].unique())

'SELECT TOP(2) [p].[Id], [p].[Avatar], [p].[CreatedBy], [p].[CreatedOn], [p].[CustomerId], [p].[Description], [p].[Leaders], [p].[ModifiedBy], [p].[ModifiedOn], [p].[Name]FROM [ProductEntity] AS [p]WHERE [p].[Id] = @__productId_0SELECT [s].[Id], [s].[Aggregation], [s].[Avatar], [s].[CreatedBy], [s].[CreatedOn], [s].[Description], [s].[Group], [s].[Leaders], [s].[ModifiedBy], [s].[ModifiedOn], [s].[Name], [s].[ProductId], [s].[Slo], [s0].[Id], [s0].[CreatedBy], [s0].[CreatedOn], [s0].[FeatureId], [s0].[ModifiedBy], [s0].[ModifiedOn], [s0].[ServiceId]FROM [ServiceEntity] AS [s]LEFT JOIN [ServiceMapEntity] AS [s0] ON [s].[Id] = [s0].[ServiceId]WHERE [s].[ProductId] = @__productId_0ORDER BY [s].[Id], [s0].[Id]SELECT [f].[Id], [f].[Avatar], [f].[CreatedBy], [f].[CreatedOn], [f].[Description], [f].[ModifiedBy], [f].[ModifiedOn], [f].[Name], [f].[ProductId], [f].[ServiceMapId], [i].[Id], [i].[Avatar], [i].[CreatedBy], [i].[CreatedOn], [i].[Description], [i].[FeatureId], [i].[ModifiedBy], [i].